## Imports:

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import matplotlib.cm as cm
from Superstuff.matching import Matching
import Superstuff.utils as ut

## cv2 implementation

In [ ]:
# load images in (left and right) # 0 -> 2 (bottle + pages) or 122 -> 126 (book + pages) or 82 -> 86 (bottle) or 66 -> 70 (mustache)
i = 335
imgR = cv2.imread("Better_Images/Image{}.png".format(i+1))
imgL = cv2.imread("Better_Images/Image{}.png".format(i))
fig, ax = plt.subplots(1,2,figsize=(12,7))
ax[0].imshow(imgL)
ax[0].set_title("Original Left Image")
ax[0].axis("off")
ax[1].imshow(imgR)
ax[1].set_title("Original Right Image")
ax[1].axis("off")
plt.show()

In [ ]:
# Setting parameters for StereoSGBM algorithm
minDisparity = 0
numDisparities = 100
blockSize = 15
disp12MaxDiff = 1
uniquenessRatio = 5
speckleWindowSize = 15
speckleRange = 10

In [ ]:
# Creating an object of StereoSGBM algorithm ()
stereo = cv2.StereoSGBM_create(minDisparity = minDisparity,
        numDisparities = numDisparities,
        blockSize = blockSize,
        disp12MaxDiff = disp12MaxDiff,
        uniquenessRatio = uniquenessRatio,
        speckleWindowSize = speckleWindowSize,
        speckleRange = speckleRange)

In [ ]:
# Calculating disparith using the StereoSGBM algorithm
disp = stereo.compute(imgL, imgR).astype(np.float32)
disp = cv2.normalize(disp,0,255,cv2.NORM_MINMAX)

In [ ]:
# Displaying the disparity map
fig, ax = plt.subplots(figsize=(20,10))

plt.imshow(disp)
plt.colorbar()
ax.imshow(disp)
ax.set_title("Disparity")
ax.axis("off")
plt.show()

## My implementation -> their disparity computation + rectification

In [ ]:
def plot_matching_features(f1, f2, im1, im2):
    image1, image2 = im1.copy(), im2.copy()
    
    for f in f1:
        image1 = cv2.circle(image1, (int(f[0]),int(f[1])), 1, (255,255,0), 2)
    for f in f2:
        image2 = cv2.circle(image2, (int(f[0]),int(f[1])), 1, (255,255,0), 2)
        
    fig, ax = plt.subplots(1,2, figsize=(15,10))
    
    ax[0].imshow(image1)
    ax[0].axis("off")
    ax[0].set_title("Left Image")
    ax[1].imshow(image2)
    ax[1].axis("off")
    ax[1].set_title("Right Image")
    plt.show()
    
def plot_shift_features(f1,f2,im1):
    # from left image we show change to points in right
    image1 = im1.copy()
    
    for l, r in zip(f1, f2):
        image1 = cv2.line(image1, (int(l[0]),int(l[1])), (int(r[0]),int(r[1])), (255,0,0), 2)
    
    plt.figure(figsize=(12,7))
    plt.imshow(image1)
    plt.axis("off")
    plt.title("Change in points from left to right image")
    plt.show()

In [ ]:
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(imgL,None)
kp2, des2 = sift.detectAndCompute(imgR,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)


pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

In [ ]:
plot_matching_features(pts1, pts2, imgL, imgR)
plot_shift_features(pts1,pts2,imgL)

In [ ]:
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)

F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

In [ ]:
plot_matching_features(pts1, pts2, imgL, imgR)
plot_shift_features(pts1,pts2,imgL)

In [ ]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c,w = img1.shape
    #img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    #img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

In [ ]:
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(imgL,imgR,lines1,pts1,pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(imgL,imgR,lines2,pts2,pts1)

fig, ax = plt.subplots(1,2, figsize=(15,10))

ax[0].imshow(img5)
ax[0].axis("off")
ax[0].set_title("Left Image")
ax[1].imshow(img3)
ax[1].axis("off")
ax[1].set_title("Right Image")
plt.show()

In [ ]:
retval, H1, H2 = cv2.stereoRectifyUncalibrated(points1=pts1, points2=pts2, F=F, imgSize=(1280,720))

In [ ]:
# warp images based off homography matrices: new img1 coords = H1 x (old img1 coords)
print("Perspective Warping Images:")
left_rectified = cv2.warpPerspective(img5, H1, (1280,720))
right_rectified = cv2.warpPerspective(img3, H2, (1280,720))

left_rec_nolines = cv2.warpPerspective(imgL, H1, (1280,720))
right_rec_nolines = cv2.warpPerspective(imgR, H2, (1280,720))

fig, ax = plt.subplots(1,2,figsize=(12,7))
ax[0].imshow(left_rectified)
ax[0].set_title("Left Rectified Image")
ax[1].imshow(right_rectified)
ax[1].set_title("Right Rectified Image")
plt.show()

fig, ax = plt.subplots(1,2,figsize=(12,7))
ax[0].imshow(left_rec_nolines)
ax[0].set_title("Left Rectified Image")
ax[1].imshow(right_rec_nolines)
ax[1].set_title("Right Rectified Image")
plt.show()

In [ ]:
# Calculating disparith using the StereoSGBM algorithm
disp3 = stereo.compute(left_rec_nolines, right_rec_nolines).astype(np.float32)
disp3 = cv2.normalize(disp3,0,255,cv2.NORM_MINMAX)

# Displaying the disparity map
fig, ax = plt.subplots(figsize=(20,10))

plt.imshow(disp3)
plt.colorbar()
ax.imshow(disp3)
ax.set_title("Disparity")
ax.axis("off")
plt.show()

## My implementation -> my disparity calculation

In [ ]:
# sum of absolute difference 
def s_abs_diff(pixel_vals_1, pixel_vals_2):

    if pixel_vals_1.shape != pixel_vals_2.shape:
        return -1
    return np.sum(abs(pixel_vals_1 - pixel_vals_2))

# compare left block of pixels with multiple blocks from the right image using 
def window_compare(y, x, left_local, right, window_size=5):
    
    local_window = 100
    
    # get search range for the right image
    x_min = max(0, x - local_window)
    x_max = min(right.shape[1], x + local_window)
    min_sad = None
    index_min = None
    first = True
    
    # only compare to points in the x row as we have reduced search space
    for x in range(x_min, x_max):
        right_local = right[y: y+window_size,x: x+window_size]
        sad = s_abs_diff(left_local, right_local)
        if first:
            min_sad = sad
            index_min = (y, x)
            first = False
        else:
            if sad < min_sad:
                min_sad = sad
                index_min = (y, x)

    return index_min

#disparity calculation on the recified images
def disparity_calc(image_left, image_right):
    
    window = 5

    left = np.asarray(image_left)
    right = np.asarray(image_right)
    
    left = left.astype(int)
    right = right.astype(int)
    
    if left.shape != right.shape:
        print("Image Shapes do not match!!")
      
    h, w, g = left.shape
    
    disparity = np.zeros((h, w))
    # going over each pixel
    for y in range(window, h-window):
        for x in range(window, w-window):
            left_local = left[y:y + window, x:x + window]
            index_min = window_compare(y, x, left_local, right, window_size = window)
            disparity[y, x] = abs(index_min[1] - x)
            
    plt.imshow(disparity, cmap='jet')
    plt.colorbar()
    plt.title('Disparity Plot Heat')
    plt.show()
    
    return disparity

In [ ]:
disparity = disparity_calc(left_rec_nolines, right_rec_nolines)
disparity = cv2.normalize(disparity,0,255,cv2.NORM_MINMAX)

In [ ]:
# Displaying the disparity map
fig, ax = plt.subplots(figsize=(20,10))

plt.imshow(disparity)
plt.colorbar()
ax.imshow(disparity)
ax.set_title("Disparity")
ax.axis("off")
plt.show()

## someone elses code:

In [ ]:
pair_number = 0
imgR = cv2.imread("3D_images/ImagePair{}-{}.png".format(pair_number, 1))
imgL = cv2.imread("3D_images/ImagePair{}-{}.png".format(pair_number, 0))
real_depth = cv2.imread("3D_images/ImagePair{}-{}.png".format(pair_number, 2))
col_img = cv2.imread("3D_images/ImagePair{}-{}.png".format(pair_number, 3))
col_img = cv2.cvtColor(col_img, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1,4,figsize=(20,10))
ax[0].imshow(imgL)
ax[0].set_title("Original Left Image")
ax[0].axis("off")
ax[1].imshow(imgR)
ax[1].set_title("Original Right Image")
ax[1].axis("off")
ax[2].imshow(real_depth)
ax[2].set_title("Real Depth")
ax[2].axis("off")
ax[3].imshow(col_img)
ax[3].set_title("Colour Image")
ax[3].axis("off")
plt.show()

In [ ]:
imgL_c = imgL.copy()
imgR_c = imgR.copy()

In [ ]:
# Initiate SIFT detector
sift = cv2.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(imgL, None)
kp2, des2 = sift.detectAndCompute(imgR, None)

In [ ]:
# Visualize keypoints
imgSift = cv2.drawKeypoints(imgL, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(imgSift)
plt.axis("off")
plt.show()

In [ ]:
# Match keypoints in both images
# Based on: https://docs.opencv.org/master/dc/dc3/tutorial_py_matcher.html
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Keep good matches: calculate distinctive image features
# Lowe, D.G. Distinctive Image Features from Scale-Invariant Keypoints. International Journal of Computer Vision 60, 91–110 (2004). https://doi.org/10.1023/B:VISI.0000029664.99615.94
# https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf
matchesMask = [[0, 0] for i in range(len(matches))]
good = []
pts1 = []
pts2 = []

for i, (m, n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        # Keep this keypoint pair
        matchesMask[i] = [1, 0]
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

In [ ]:
# Draw the keypoint matches between both pictures
# Still based on: https://docs.opencv.org/master/dc/dc3/tutorial_py_matcher.html
draw_params = dict(matchColor=(0, 255, 0),
                   singlePointColor=(255, 0, 0),
                   matchesMask=matchesMask[300:500],
                   flags=cv2.DrawMatchesFlags_DEFAULT)

keypoint_matches = cv2.drawMatchesKnn(
    imgL, kp1, imgR, kp2, matches[200:400], None, **draw_params)
plt.imshow(keypoint_matches)

In [ ]:
# Calculate the fundamental matrix for the cameras
# https://docs.opencv.org/master/da/de9/tutorial_py_epipolar_geometry.html
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
fundamental_matrix, inliers = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

# We select only inlier points
pts1 = pts1[inliers.ravel() == 1]
pts2 = pts2[inliers.ravel() == 1]

In [ ]:
# Visualize epilines
# Adapted from: https://docs.opencv.org/master/da/de9/tutorial_py_epipolar_geometry.html
def drawlines(img1src, img2src, lines, pts1src, pts2src):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r, c, h = img1src.shape
    #img1color = cv2.cvtColor(img1src, cv2.COLOR_GRAY2BGR)
    #img2color = cv2.cvtColor(img2src, cv2.COLOR_GRAY2BGR)
    # Edit: use the same random seed so that two images are comparable!
    np.random.seed(0)
    for r, pt1, pt2 in zip(lines, pts1src, pts2src):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2]/r[1]])
        x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
        img1color = cv2.line(img1src, (x0, y0), (x1, y1), color, 1)
        img1color = cv2.circle(img1src, tuple(pt1), 5, color, -1)
        img2color = cv2.circle(img2src, tuple(pt2), 5, color, -1)
    return img1color, img2color


# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(
    pts2.reshape(-1, 1, 2), 2, fundamental_matrix)
lines1 = lines1.reshape(-1, 3)
img5, img6 = drawlines(imgL, imgR, lines1, pts1, pts2)

# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(
    pts1.reshape(-1, 1, 2), 1, fundamental_matrix)
lines2 = lines2.reshape(-1, 3)
img3, img4 = drawlines(imgR, imgL, lines2, pts2, pts1)

plt.subplot(121), plt.imshow(img5)
plt.subplot(122), plt.imshow(img3)
plt.suptitle("Epilines in both images")
plt.show()

In [ ]:
# Stereo rectification (uncalibrated variant)
# Adapted from: https://stackoverflow.com/a/62607343
h1, w1, z1 = imgL.shape
h2, w2, z2 = imgR.shape
_, H1, H2 = cv2.stereoRectifyUncalibrated(
    np.float32(pts1), np.float32(pts2), fundamental_matrix, imgSize=(w1, h1))

In [ ]:
# Undistort (rectify) the images and save them
# Adapted from: https://stackoverflow.com/a/62607343
img1_rectified = cv2.warpPerspective(imgL, H1, (w1, h1))
img2_rectified = cv2.warpPerspective(imgR, H2, (w2, h2))

img1_rectified_clean = cv2.warpPerspective(imgL_c, H1, (w1, h1))
img2_rectified_clean = cv2.warpPerspective(imgR_c, H2, (w2, h2))

In [ ]:
# Draw the rectified images
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes[0, 0].imshow(img1_rectified, cmap="gray")
axes[0, 1].imshow(img2_rectified, cmap="gray")
axes[1, 0].imshow(img1_rectified_clean, cmap="gray")
axes[1, 1].imshow(img2_rectified_clean, cmap="gray")
plt.suptitle("Rectified images")
plt.show()

In [ ]:
# Matched block size. It must be an odd number >=1 . Normally, it should be somewhere in the 3..11 range.
block_size = 10
min_disp = -64
max_disp = 64
# Maximum disparity minus minimum disparity. The value is always greater than zero.
# In the current implementation, this parameter must be divisible by 16.
num_disp = max_disp - min_disp
# Margin in percentage by which the best (minimum) computed cost function value should "win" the second best value to consider the found match correct.
# Normally, a value within the 5-15 range is good enough
uniquenessRatio = 5
# Maximum size of smooth disparity regions to consider their noise speckles and invalidate.
# Set it to 0 to disable speckle filtering. Otherwise, set it somewhere in the 50-200 range.
speckleWindowSize = 200
# Maximum disparity variation within each connected component.
# If you do speckle filtering, set the parameter to a positive value, it will be implicitly multiplied by 16.
# Normally, 1 or 2 is good enough.
speckleRange = 2
disp12MaxDiff = 0

left_matcher = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    uniquenessRatio=uniquenessRatio,
    speckleWindowSize=speckleWindowSize,
    speckleRange=speckleRange,
    disp12MaxDiff=disp12MaxDiff,
    P1=8 * 1 * block_size * block_size,
    P2=32 * 1 * block_size * block_size,
)

right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)

grayLeft = cv2.cvtColor(img1_rectified_clean, cv2.COLOR_BGR2GRAY)
grayRight = cv2.cvtColor(img2_rectified_clean, cv2.COLOR_BGR2GRAY)

left_disp = left_matcher.compute(grayLeft, grayRight)
right_disp = right_matcher.compute(grayRight, grayLeft)

sigma = 10
lmbda = 10
wls_filter = cv2.ximgproc.createDisparityWLSFilter(left_matcher)
wls_filter.setLambda(lmbda)
wls_filter.setSigmaColor(sigma)
filtered_disp = wls_filter.filter(left_disp, grayLeft, disparity_map_right=right_disp)

# Displaying the disparity map
fig, ax = plt.subplots(figsize=(20,10))
plt.imshow(filtered_disp, cmap='jet')
plt.colorbar()
ax.imshow(filtered_disp, cmap='jet')
ax.set_title("Disparity")
ax.axis("off")
plt.show()

In [ ]:
grayLeft_c = cv2.cvtColor(imgL_c, cv2.COLOR_BGR2GRAY)
grayRight_c = cv2.cvtColor(imgR_c, cv2.COLOR_BGR2GRAY)

left_disp = left_matcher.compute(grayLeft_c, grayRight_c)
right_disp = right_matcher.compute(grayRight_c, grayLeft_c)

sigma = 4
lmbda = 100
wls_filter = cv2.ximgproc.createDisparityWLSFilter(left_matcher)
wls_filter.setLambda(lmbda)
wls_filter.setSigmaColor(sigma)
filtered_disp = wls_filter.filter(left_disp, grayLeft_c, disparity_map_right=right_disp)

# Displaying the disparity map
fig, ax = plt.subplots(figsize=(20,10))
plt.imshow(filtered_disp, cmap='jet')
plt.colorbar()
ax.imshow(filtered_disp, cmap='jet')
ax.set_title("Disparity")
ax.axis("off")
plt.show()